<a href="https://colab.research.google.com/github/tbazzi/Data-Science---Fullstack-Bootcamp/blob/master/S9_4_Spark_MLlib_Classification_SOLUTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Chargement des données

1. connectez vous au compte de stockage et importez les datasets du challenge yelp

In [0]:
spark.conf.set(
  "fs.azure.account.key.storagestudent.blob.core.windows.net", 
  "pH3rgal+XcwJXc3hQEYEAE+dMBo6YzhKnb4iYQNlTZ9lXaxe8RWmZwVPMF1j2V5zwBnBZ/iNu8JoFgApOxdn4Q=="
)

In [0]:
datasets = {
  dataset: spark.read.load( 
    "wasbs://default@storagestudent.blob.core.windows.net/datasets/S8-4/Exo/restaurant-data-with-consumer-ratings/{0}.csv".format(dataset), 
    format="csv",
    header="true"
  )
  for dataset in [
    "chefmozaccepts", 
    "chefmozcuisine", 
    "chefmozhours4", 
    "chefmozparking", 
    "geoplaces2", 
    "rating_final", 
    "usercuisine", 
    "userpayment", 
    "userprofile"
  ]
} 

In [0]:
datasets

Out[ 8 ]: 
{'chefmozcuisine': DataFrame[placeID: string, Rcuisine: string],
 'userprofile': DataFrame[userID: string, latitude: string, longitude: string, smoker: string, drink_level: string, dress_preference: string, ambience: string, transport: string, marital_status: string, hijos: string, birth_year: string, interest: string, personality: string, religion: string, activity: string, color: string, weight: string, budget: string, height: string],
 'chefmozhours4': DataFrame[placeID: string, hours: string, days: string],
 'userpayment': DataFrame[userID: string, Upayment: string],
 'usercuisine': DataFrame[userID: string, Rcuisine: string],
 'geoplaces2': DataFrame[placeID: string, latitude: string, longitude: string, the_geom_meter: string, name: string, address: string, city: string, state: string, country: string, fax: string, zip: string, alcohol: string, smoking_area: string, dress_code: string, accessibility: string, price: string, url: string, Rambience: string, franchise: string, area: string, other_services: string],
 'chefmozaccepts': DataFrame[placeID: string, Rpayment: string],
 'rating_final': DataFrame[userID: string, placeID: string, rating: string, food_rating: string, service_rating: string],
 'chefmozparking': DataFrame[placeID: string, parking_lot: string]}

2. créez trois objets rating, user, place, contenant respectivement les datasets rating_final, user_profile et geoplaces2

In [0]:
rating = datasets["rating_final"]
user = datasets["userprofile"]
place = datasets["geoplaces2"]

#1. Cleaning des données (Recodage du label, typage et jointures)

3. dans rating, transformez la variables rating pour qu'elle vale 1 lorsque rating vaut 2 et 0 le reste du temps

In [0]:
from pyspark.sql import functions as F

rating_cleaned = rating.select(
  F.col("userID"),
  F.col("placeID"),
  F.when(F.col("rating") == "2", 1).otherwise(0).alias("label")
)


display(rating_cleaned)

userID,placeID,label
U1077,135085,1
U1077,135038,1
U1077,132825,1
U1077,135060,0
U1068,135104,0
U1068,132740,0
U1068,132663,0
U1068,132732,0
U1068,132630,0
U1067,132584,1


4. Formattez la table user afin que les colonnes soient du type adéquat

In [0]:
from pyspark.sql.types import *

user_cleaned = user.select(
  F.col("userID"),
  #F.col("latitude"),
  #F.col("longitude"),
  F.col("smoker").cast(BooleanType()),
  F.col("drink_level"),
  F.col("dress_preference"),
  F.col("ambience"),
  F.col("transport"),
  F.col("marital_status"),
  F.col("hijos"),
  F.col("birth_year").cast(FloatType()),
  F.col("interest"),
  F.col("personality"),
  F.col("religion"),
  F.col("activity"),
  F.col("color"),
  F.col("weight").cast(FloatType()),
  F.col("budget"),
  F.col("height").cast(FloatType()),
)

display(user_cleaned)

userID,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height
U1001,false,abstemious,informal,family,on foot,single,independent,1989.0,variety,thrifty-protector,none,student,black,69.0,medium,1.77
U1002,false,abstemious,informal,family,public,single,independent,1990.0,technology,hunter-ostentatious,Catholic,student,red,40.0,low,1.87
U1003,false,social drinker,formal,family,public,single,independent,1989.0,none,hard-worker,Catholic,student,blue,60.0,low,1.69
U1004,false,abstemious,informal,family,public,single,independent,1940.0,variety,hard-worker,none,professional,green,44.0,medium,1.53
U1005,false,abstemious,no preference,family,public,single,independent,1992.0,none,thrifty-protector,Catholic,student,black,65.0,medium,1.69
U1006,true,social drinker,no preference,friends,car owner,single,independent,1989.0,variety,hard-worker,none,student,blue,75.0,medium,1.8
U1007,false,casual drinker,informal,solitary,public,single,independent,1989.0,variety,thrifty-protector,Catholic,student,purple,60.0,low,1.59
U1008,false,social drinker,formal,solitary,public,single,independent,1989.0,technology,hard-worker,Catholic,student,green,68.0,low,1.72
U1009,false,abstemious,formal,family,on foot,single,kids,1991.0,variety,thrifty-protector,Catholic,student,green,75.0,medium,1.78
U1010,false,social drinker,no preference,friends,car owner,married,kids,1987.0,technology,hard-worker,none,student,green,40.0,medium,1.67


5. Même exercice avec place

In [0]:
place_cleaned = place.select(
  F.col("placeID"),
  #F.col("latitude"),
  #F.col("longitude"),
  #F.col("the_geom_meter"),
  #F.col("name"),
  #F.col("address"),
  #F.col("city"),
  #F.col("state"),
  #F.col("country"),
  #F.col("fax"),
  #F.col("zip"),
  F.col("alcohol"),
  F.col("smoking_area"),
  F.col("dress_code"),
  F.col("accessibility"),
  F.col("price"),
  #F.col("url"),
  F.col("Rambience"),
  F.col("franchise"),
  F.col("area"),
  F.col("other_services"),
)

display(place_cleaned)

placeID,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services
134999,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,closed,none
132825,No_Alcohol_Served,none,informal,completely,low,familiar,f,open,none
135106,Wine-Beer,only at bar,informal,partially,medium,familiar,f,open,none
132667,No_Alcohol_Served,none,informal,completely,low,familiar,t,closed,none
132613,No_Alcohol_Served,permitted,informal,completely,medium,familiar,t,closed,none
135040,Wine-Beer,none,informal,no_accessibility,high,familiar,f,closed,none
132732,No_Alcohol_Served,none,casual,completely,low,familiar,f,open,none
132875,Wine-Beer,section,informal,no_accessibility,high,familiar,t,open,Internet
132609,No_Alcohol_Served,not permitted,informal,completely,low,quiet,t,closed,none
135082,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,closed,none


6. Effectuez une jointure entre les tables user et rating préalablement nettoyées

In [0]:
training_basis = rating_cleaned.join(user_cleaned, "userID" , "left").join(place_cleaned, "placeID", "left").dropna()

display(training_basis)

placeID,userID,label,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services
135085,U1077,1,false,social drinker,elegant,family,public,married,kids,1987.0,technology,thrifty-protector,Catholic,student,blue,65.0,medium,1.71,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,f,closed,none
135038,U1077,1,false,social drinker,elegant,family,public,married,kids,1987.0,technology,thrifty-protector,Catholic,student,blue,65.0,medium,1.71,No_Alcohol_Served,section,informal,no_accessibility,medium,familiar,f,closed,none
132825,U1077,1,false,social drinker,elegant,family,public,married,kids,1987.0,technology,thrifty-protector,Catholic,student,blue,65.0,medium,1.71,No_Alcohol_Served,none,informal,completely,low,familiar,f,open,none
135060,U1077,0,false,social drinker,elegant,family,public,married,kids,1987.0,technology,thrifty-protector,Catholic,student,blue,65.0,medium,1.71,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,closed,none
135104,U1068,0,false,casual drinker,informal,friends,public,single,independent,1988.0,technology,thrifty-protector,Catholic,student,blue,72.0,low,1.57,Full_Bar,not permitted,informal,completely,medium,familiar,t,closed,variety
132740,U1068,0,false,casual drinker,informal,friends,public,single,independent,1988.0,technology,thrifty-protector,Catholic,student,blue,72.0,low,1.57,No_Alcohol_Served,permitted,informal,completely,low,familiar,f,open,none
132663,U1068,0,false,casual drinker,informal,friends,public,single,independent,1988.0,technology,thrifty-protector,Catholic,student,blue,72.0,low,1.57,No_Alcohol_Served,none,informal,completely,low,familiar,f,closed,none
132732,U1068,0,false,casual drinker,informal,friends,public,single,independent,1988.0,technology,thrifty-protector,Catholic,student,blue,72.0,low,1.57,No_Alcohol_Served,none,casual,completely,low,familiar,f,open,none
132630,U1068,0,false,casual drinker,informal,friends,public,single,independent,1988.0,technology,thrifty-protector,Catholic,student,blue,72.0,low,1.57,No_Alcohol_Served,none,informal,completely,low,familiar,f,closed,none
132584,U1067,1,false,abstemious,no preference,family,public,single,independent,1987.0,technology,thrifty-protector,Christian,student,green,92.0,medium,1.73,No_Alcohol_Served,not permitted,informal,completely,medium,familiar,t,closed,none


#2. Pre processing pour MLlib

7. Appliquez les manipulations de preprocessing sur la table jointe afin de la préparer à l'entraînement d'un modèle dans spark ML, vous devrez choisir comme variable cible rating et le choix des variables explicatives vous revient

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier


features = ["smoker", "drink_level", "dress_preference", "ambience", "transport", "marital_status", "hijos", "birth_year", "interest", "personality", "religion", "activity", "color", "weight", "budget", "height", "alcohol", "smoking_area", "dress_code", "accessibility", "price", "Rambience", "franchise", "area", "other_services"]

categorical_features = ["drink_level", "dress_preference", "ambience", "transport", "marital_status", "hijos", "interest", "personality", "religion", "activity", "color", "budget", "alcohol", "smoking_area", "dress_code", "accessibility", "price", "Rambience", "franchise", "area", "other_services"]


string_indexers = [StringIndexer(inputCol=col, outputCol=col + "Index", handleInvalid="skip") for col in categorical_features]
one_hot_encoder = OneHotEncoderEstimator(inputCols=[col+"Index" for col in categorical_features], outputCols=[col+"Vec" for col in categorical_features])
vector_assembler = VectorAssembler(inputCols=[col+"Vec" for col in categorical_features] + ["smoker", "weight", "height"], outputCol="features")

pre_processing_pipeline = Pipeline(stages = [*string_indexers, one_hot_encoder, vector_assembler])

#3. Entrainement du modèle

8. Séparez vos données en un échantillon train et test

In [0]:
pre_processing_pipeline_fitted = pre_processing_pipeline.fit(training_basis)
training_basis_cleaned = pre_processing_pipeline_fitted.transform(training_basis)
train, test = training_basis_cleaned.randomSplit([0.7, 0.3], 100)

9. Entraînez un random forest classifier ou un autre modèle de votre choix afin de prédire la valeur de rating. Evaluez ensuite les performances de ce modèle.
Vous pouvez bien entendu faire une grid search afin de déterminer des valeurs bien choisies des hyper paramètres.

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

rf = RandomForestClassifier(labelCol="label", featuresCol="features")

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")

paramGrid = ParamGridBuilder()\
             .addGrid(rf.maxDepth, [2, 5, 10])\
             .addGrid(rf.maxBins, [8, 16, 32])\
             .addGrid(rf.numTrees, [1, 5, 10])\
             .build()
 
cv_rf = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cv_rf_fitted = cv_rf.fit(train)

#4. Evaluation et analyse

10. Evaluez maintenant le modèle à l'aide des techniques habituelles.

In [0]:
bestModel = cv_rf_fitted.bestModel

bestModelParameters = {
  key.name: value
  for key, value in bestModel.extractParamMap().items()
  if key.name in ["maxDepth", "maxBins", "numTrees"]
}

print(bestModelParameters)

{'maxBins': 16, 'maxDepth': 10, 'numTrees': 5}

In [0]:
predictions = bestModel.transform(test)

evaluator.evaluate(predictions)

Out[ 17 ]: 0.6847773972602738

In [0]:
predictions.select(F.col("label"), F.col("prediction").cast(StringType()).alias("prediction"))

Out[ 18 ]: DataFrame[label: int, prediction: string]

In [0]:
total_count = predictions.count()

cf_count = predictions.groupBy("prediction").pivot("label").count() 

display(cf_count)

prediction,0,1
0.0,149,80
1.0,51,66


In [0]:
cf = cf_count.select(
  F.col("prediction"),
  (F.col("0") / total_count).alias("0"),
  (F.col("1") / total_count).alias("1")
)

display(cf)

prediction,0,1
0.0,0.430635838150289,0.23121387283236994
1.0,0.14739884393063585,0.1907514450867052
